In [ ]:
import os
import uuid

import holoviews as hv
import hvplot.pandas
import numpy as np
import pandas as pd

from holonote.annotate import Annotator, SQLiteDB

hv.extension("bokeh")
hv.opts.defaults(hv.opts.Rectangles(apply_ranges=False))

In [ ]:
speed_data = pd.read_parquet("./assets/example.parquet")
speed_curve = speed_data.hvplot("TIME", "SPEED")
speed_curve = hv.Curve(
    (np.arange("2005-02", "2005-03", dtype="datetime64[D]"), range(28)), kdims=["TIME"]
)
speed_curve

In [ ]:
xvals = np.linspace(-4, 0, 202)
yvals = np.linspace(4, 0, 202)
xs, ys = np.meshgrid(xvals, yvals)

alpha, beta = 1, 0
data = np.sin(((ys / alpha) ** alpha + beta) * xs)

image = hv.Image(data, kdims=["A", "B"]).opts(cmap="greens")
image

# Main branch implementation
This is how it works on the main branch:

# Current implementation - part 1

Based on Jean-Lucs vision of passing in an element

The is how it works on "my" branch. 

This is not 1-for-1 for with main as it returns two rows, where main returns one row with annotations: 


<div>
<img src="multi_example.png" width="1000"/>
</div>

In [ ]:
connector = SQLiteDB(table_name="Idecided")

# Check fields match when shared, (clone is needed to not overlay the original element)
curve_annotator = Annotator(
    speed_curve.clone(), connector=connector, fields=["description"], init=False
)
image_annotator = Annotator(
    image, connector=connector, fields=["description"], init=False
)
curve_annotator.load()
image_annotator.load()

In [ ]:
curve_annotator.overlay() + image_annotator.overlay()

In [ ]:
# Gives problems when having multiple of the same annotations
# I have in all my other projects used an empty element (hv.Curve([]) as input element.
curve_annotator.overlay() + image_annotator.overlay()

In [ ]:
curve_annotator.set_range(np.datetime64("2005-02-13"), np.datetime64("2005-02-16"))
image_annotator.set_range(-0.25, 0.25, -0.1, 0.1)

In [ ]:
curve_annotator.add_annotation(description="Annotation 2")
image_annotator.add_annotation(description="Annotation 2")
curve_annotator.commit()
image_annotator.commit()

# Current implementation - part 2
Based on how I do things for projects: (not working with Image - so removed it for simplicity)

In [ ]:
connector = SQLiteDB(table_name="Idecided2")

curve_annotator = Annotator(
    # spec=Annotator._infer_kdim_dtypes(speed_curve),
    spec={"TIME": np.datetime64},
    connector=connector,
    fields=["description"],
)

anno = curve_annotator.overlay(hv.Curve([]))  # hack

In [ ]:
anno * speed_curve

In [ ]:
anno * speed_curve

# Simon future vision
The is how I _think_ I would like it to work

# Jean-Lucs future vision
See: https://github.com/holoviz/holonote/issues/9